# Goal

Prevent a credit card fraud: when a bad actor uses someone else's credit card information to make a purchase without their permission.

**Columns**:
- **distance_from_home** - the distance from home where the transaction happened.
- **distance_from_last_transaction** - the distance from last transaction happened.
- **ratio_to_median_purchase_price** - Ratio of purchased price transaction to median purchase price.
- **repeat_retailer** - Is the transaction happened from same retailer.
- **used_chip** - Is the transaction through chip (credit card).
- **used_pin_number** - Is the transaction happened by using PIN number.
- **online_order** - Is the transaction an online order.
- **fraud** - Is the transaction fraudulent. This is what we're trying to predict.

# Imports

## libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## data

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

---

# Exploratory Data Analysis 

## General

In [5]:
train.shape, test.shape

((18000, 9), (2000, 8))

In [6]:
train.isnull().sum()

id                                0
distance_from_home                0
distance_from_last_transaction    0
ratio_to_median_purchase_price    0
repeat_retailer                   0
used_chip                         0
used_pin_number                   0
online_order                      0
fraud                             0
dtype: int64

In [7]:
test.isnull().sum()

id                                0
distance_from_home                0
distance_from_last_transaction    0
ratio_to_median_purchase_price    0
repeat_retailer                   0
used_chip                         0
used_pin_number                   0
online_order                      0
dtype: int64

In [8]:
train

,id,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,16979,3.346870,0.620101,0.945172,1.0,0.0,0.0,1.0,0
1,12430,14.482868,0.170185,1.084836,1.0,0.0,0.0,1.0,0
2,19872,5.740898,0.167525,3.969652,1.0,0.0,0.0,1.0,0
3,18763,4.098570,0.291850,0.084129,1.0,1.0,0.0,1.0,1
4,13547,7.539580,0.294439,2.890880,1.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...
17995,11193,8.191938,0.295913,1.011287,1.0,0.0,0.0,1.0,0
17996,10245,10.098491,1.042468,3.073815,1.0,0.0,0.0,1.0,0
17997,5660,2.826653,0.093371,0.047923,1.0,1.0,0.0,0.0,0
17998,10839,0.826856,0.164881,2.827485,0.0,0.0,0.0,1.0,0


In [12]:
train_index = len(train)-1
test_index = len(train)
train_index, test_index

(17999, 18000)